In [0]:
from pyspark.sql.functions import to_date, col, round, current_timestamp
import sys
from time import time

In [0]:
sys.path.append("/Workspace/Users/faraaz_khan@epam.com/PEI/utils")

In [0]:
from utils.common_util import convert_date_columns

In [0]:
start_time = time()

In [0]:
checkpoint_path = "/Volumes/fz_catalog/landing/_checkpoints/orders_silver/"
target_table ="fz_catalog.silver.orders"

In [0]:
df = (
    spark.readStream
         .format("delta")
         .table("fz_catalog.bronze.orders")
         .drop("input_file_name", "file_modification_time")
)

In [0]:
df = convert_date_columns(df, ["order_date", "ship_date"], "d/M/yyyy")

In [0]:
df_silver = df.withColumn("profit", round(col("profit"),2))

In [0]:
df_silver = df_silver.withColumn("created_date", current_timestamp())

In [0]:
df_silver.writeStream.format("delta")\
.outputMode("append")\
.option("checkpointLocation", checkpoint_path)\
.trigger(availableNow=True)\
.toTable("fz_catalog.silver.orders")

In [0]:
end_time = time()

In [0]:
dbutils.notebook.exit(f"Run Time: {end_time - start_time} seconds")

defined pk and drop suplicates based on that
create a dmnesionl diagram fact